### Scraping du Blog : https://www.charlesbordet.com/fr/blog/

"Scraping en cascade"
- Scraper la page suivante: https://www.charlesbordet.com/fr/blog/
- Récupérer la liste des articles
- Pour chaque articles retrouver l'url vers la page dédiée à cet article (par exemple pour le 1er article l'adresse est : https://www.charlesbordet.com/fr/reactive-shiny/)
- Une fois arrivé sur cette page récupérer le sommaire
- Produire une liste d'articles comprenant 
    - Le titre
    - La description
    - Le tps de lecture
    - (Et surtout) le sommaire (modifié)


In [218]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

### Je résume dans une fonction le ***scrapping du sommaire*** (pour l'appliquer après à chaque article) :

In [239]:
def scrap_sommaire(url, sub_url):
    sub_page = requests.get(url+sub_url) 
    soup = bs(sub_page.content,'html.parser') 
    sommaire = soup.select(".toc__menu") # class concernant le Sommaire
    
    sommaire_dict = {'Sommaire':[]}
    for list in sommaire[0]:
        if list.ul:
            sommaire_dict['Sommaire'].append(list.find("a").get_text())
            for sublist in list.ul:
                sommaire_dict['Sommaire'].append(sublist.find("a").get_text())
        else:
            sommaire_dict['Sommaire'].append(list.find("a").get_text())
        
    return pd.DataFrame(sommaire_dict)

### Boucle sur **CHAQUE ARTICLE** de la page principale : on stocke le titre, l'URL, la description, le temps de lecture, ET le sommaire en format DataFrame Pandas (autre méthode possible)

In [240]:
welcome_url = "https://www.charlesbordet.com"
welcome_page = requests.get(welcome_url+"/fr/blog/")
welcome_soup = bs(welcome_page.content, 'html.parser')

liste_articles = welcome_soup.select(".list__item")             #rassemble TOUS les articles de la page principale
article_titles = welcome_soup.select(".archive__item-title")    #rassemble les titres avec url de chaque article

articles_infos = []
for i in range(len(article_titles)):    #on boucle sur le nombre d'article recensé dans la page d'accueil
    
    title = article_titles[i].find("a").get_text()
    description = liste_articles[i].select(".archive__item-excerpt")[0].text
    reading_time = liste_articles[i].select(".page__meta")[0].get_text().strip()
    sub_url = article_titles[i].find("a").get("href")
    sommaire = scrap_sommaire(welcome_url, sub_url) # fonction 'scrap_sommaire' qui boucle sur les url
    articles_infos.append({'Title': title,
                           'Description': description,
                           'Temps de lecture': reading_time,
                           'URL': sub_url,
                           'Sommaire': sommaire})

articles_infos #liste de dictionnaires contenant 'Titre/Description/url'

[{'Title': 'Le guide complet pour comprendre la réactivité en Shiny\n',
  'Description': 'Comprendre la réactivité en Shiny est fondamental pour être capable de créer des applications complexes mais performantes et faciles à maintenir.\n',
  'Temps de lecture': '51 minutes de lecture',
  'URL': '/fr/reactive-shiny/',
  'Sommaire':                                              Sommaire
  0                          C’est quoi la réactivité ?
  1    Les fonctions de réactivité : req, isolate, etc.
  2                              La fonction reactive()
  3                         La fonction eventReactive()
  4                               La fonction observe()
  5                          La fonction observeEvent()
  6                        La fonction reactiveValues()
  7                               La fonction isolate()
  8                                   La fonction req()
  9         Mes petites astuces autour de la réactivité
  10       Comment forcer la réactivité avec Sys.time

In [246]:
articles_infos

[{'Title': 'Le guide complet pour comprendre la réactivité en Shiny\n',
  'Description': 'Comprendre la réactivité en Shiny est fondamental pour être capable de créer des applications complexes mais performantes et faciles à maintenir.\n',
  'Temps de lecture': '51 minutes de lecture',
  'URL': '/fr/reactive-shiny/',
  'Sommaire':                                              Sommaire
  0                          C’est quoi la réactivité ?
  1    Les fonctions de réactivité : req, isolate, etc.
  2                              La fonction reactive()
  3                         La fonction eventReactive()
  4                               La fonction observe()
  5                          La fonction observeEvent()
  6                        La fonction reactiveValues()
  7                               La fonction isolate()
  8                                   La fonction req()
  9         Mes petites astuces autour de la réactivité
  10       Comment forcer la réactivité avec Sys.time

In [248]:
articles_infos[0]['Sommaire']

,Sommaire
0,C’est quoi la réactivité ?
1,"Les fonctions de réactivité : req, isolate, etc."
2,La fonction reactive()
3,La fonction eventReactive()
4,La fonction observe()
5,La fonction observeEvent()
6,La fonction reactiveValues()
7,La fonction isolate()
8,La fonction req()
9,Mes petites astuces autour de la réactivité


In [214]:
url + articles_infos[0]['url']

'https://www.charlesbordet.com/fr/reactive-shiny/'

In [215]:
test = articles_infos[0] #reactive-shiny
sub_page = requests.get(url+test['url']) 
soup = bs(sub_page.content,'html.parser') 
sommaire = soup.select(".toc__menu") # class concernant le Sommaire

In [216]:
test

{'Title': 'Le guide complet pour comprendre la réactivité en Shiny\n',
 'Description': 'Comprendre la réactivité en Shiny est fondamental pour être capable de créer des applications complexes mais performantes et faciles à maintenir.\n',
 'Temps de lecture': '51 minutes de lecture',
 'url': '/fr/reactive-shiny/'}

In [201]:
sommaire[0]

<ul class="toc__menu"><li><a href="#c-est-quoi-la-reactivite">C’est quoi la réactivité ?</a></li><li><a href="#les-fonctions-de-reactivite-req-isolate-etc">Les fonctions de réactivité : req, isolate, etc.</a><ul><li><a href="#la-fonction-reactive">La fonction reactive()</a></li><li><a href="#la-fonction-eventreactive">La fonction eventReactive()</a></li><li><a href="#la-fonction-observe">La fonction observe()</a></li><li><a href="#la-fonction-observeevent">La fonction observeEvent()</a></li><li><a href="#la-fonction-reactivevalues">La fonction reactiveValues()</a></li><li><a href="#la-fonction-isolate">La fonction isolate()</a></li><li><a href="#la-fonction-req">La fonction req()</a></li></ul></li><li><a href="#mes-petites-astuces-autour-de-la-reactivite">Mes petites astuces autour de la réactivité</a><ul><li><a href="#comment-forcer-la-reactivite-avec-sys-time">Comment forcer la réactivité avec Sys.time()</a></li><li><a href="#comment-accelerer-un-calcul-avec-bindcache">Comment accélé

### Récupération du sommaire 

In [156]:
sommaire_dict = {'Sommaire':[]}
print('Sommaire')
for list in sommaire[0]:
    if list.ul:
        print('liste avec sous-liste :',list.find("a").get_text())
        sommaire_dict['Sommaire'].append(list.find("a").get_text())
        for sublist in list.ul:
            print('* :', sublist.find("a").get_text())
            sommaire_dict['Sommaire'].append(sublist.find("a").get_text())
    else:
        print(list.find("a").get_text(),'\n')
        sommaire_dict['Sommaire'].append(list.find("a").get_text())

Sommaire
C’est quoi la réactivité ? 

liste avec sous-liste : Les fonctions de réactivité : req, isolate, etc.
* : La fonction reactive()
* : La fonction eventReactive()
* : La fonction observe()
* : La fonction observeEvent()
* : La fonction reactiveValues()
* : La fonction isolate()
* : La fonction req()
liste avec sous-liste : Mes petites astuces autour de la réactivité
* : Comment forcer la réactivité avec Sys.time()
* : Comment accélérer un calcul avec bindCache
* : Comment avoir plusieurs inputs dans un observeEvent
Cas d'usage: Dans un shinydashboard 

liste avec sous-liste : Cas d'usage: Les filtres interdépendants
* : Solution 1 : Supprimer l’interdépendance
* : Solution 2 : Ajouter un bouton
* : Solution 3 : Filtres emboîtés avec garantie de changement
* : Solution 4 : Filtres emboîtés sans garantie de changement
* : Solution 5 : Si les filtres sont rapides à calculer


In [158]:
df_0 = pd.DataFrame(sommaire_dict)

In [159]:
df_0

,Sommaire
0,C’est quoi la réactivité ?
1,"Les fonctions de réactivité : req, isolate, etc."
2,La fonction reactive()
3,La fonction eventReactive()
4,La fonction observe()
5,La fonction observeEvent()
6,La fonction reactiveValues()
7,La fonction isolate()
8,La fonction req()
9,Mes petites astuces autour de la réactivité
